#  Code for 2D staggered Lagrangian method in Cylindrical coordinates

In [1]:
struct Const
    Y0 ::Float64 #Yielding strength
    ρ0 ::Float64 
    Γ0 ::Float64
    μ  ::Float64
    a0 ::Float64
    s0 ::Float64
end


In [2]:
mutable struct Var
    r::Array{Float64,1}
    z::Array{Float64,1}
    u::Array{Float64,1}
    v::Array{Float64,1}
    ρ::Array{Float64,1}
    p::Array{Float64,1}
    s::Array{Float64,2} # σ[1] = σrr σ[2] = σzz σ[3] = σrz
end    

In [3]:
struct prb
    tt::Float64
    sf::Float64
end

## Control equations

一般形式下，弹塑性流体控制方程为：

质量方程：
\begin{equation}
  \frac{dm}{dt}=0
  \end{equation}
动量方程：
\begin{equation}
  \frac{d\rho \mathbf{u}}{dt}= \nabla \cdot  \mathbf{\Pi} 
\end{equation}

能量方程：
\begin{equation}
 \rho \frac{de}{dt}= \mathbf{\sigma} \cdot \mathbf{\varepsilon}
\end{equation}









## Cylindrical coordinate
运动方程：
\begin{equation}
  \frac{d(r,z)}{dt}=(u,v)
\end{equation}

质量守恒：
\begin{equation}\label{eq:38}
  \frac{dm}{dt}=0
\end{equation}

动量守恒方程：
\begin{equation}\label{eq:3}
  \begin{array}{l}
	\displaystyle \rho \frac{du}{dt}=\frac{\partial \sigma _{rr}}{\partial r}+ \frac{\partial s_{rz}}{\partial z} + \frac{\sigma_{rr}-\sigma_{\theta \theta}}{r}\\[0.3cm]
  \displaystyle \rho \frac{dv}{dt}=\frac{\partial \sigma _{zz}}{\partial z}+ \frac{\partial s_{rz}}{\partial r} + \frac{s_{rz}}{r}\\
  \end{array}
  \end{equation}

能量守恒方程：
\begin{equation}
  \begin{array}{l}
	\displaystyle \rho \frac{de}{dt}=\sigma _{rr}\frac{\partial u}{\partial r}+\sigma_{zz}\frac{\partial v}{\partial z}+\sigma_{\theta \theta}\frac{u}{r}+s_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle = -p\frac{dV}{dt}+ s_{rr}\frac{\partial u}{\partial r}+s_{zz}\frac{\partial v}{\partial z}+s_{\theta\theta}\frac{u}{r}+s_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)\\
	\end{array}
	\end{equation}

本构方程：
\begin{equation}
  \begin{array}{l}
	\displaystyle \frac{ds_{rr}}{dt}=2\mu \left(\frac{\partial u}{\partial r}-\frac{1}{3}\nabla \cdot \vec{u} \right) +s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{zz}}{dt}=2\mu \left(\frac{\partial v}{\partial z}-\frac{1}{3}\nabla \cdot \vec{u} \right) -s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
    \displaystyle \frac{ds_{\theta\theta}}{dt}=2\mu \left(\frac{u}{r}-\frac{1}{3}\nabla \cdot \vec{u} \right)\\[0.3cm]
	\displaystyle \frac{ds_{rz}}{dt}=\mu \left(\frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right) + \frac{s_{rr}-s_{zz}}{2}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\
  \end{array}
  \end{equation}

Von Mises屈服条件：
\begin{equation}
  \frac{3}{2}(s_{rr}^2+s_{zz}^2+s_{\theta \theta}^2+2s_{xy}^2)\le (Y^0)^2
\end{equation}


$$ s_{\theta\theta} = - (s_{rr} +s_{zz}) $$

## Compatible Staggered Discretization

<img src="Grid.png" width = "800" height = "400" div align=center /> 

<img src="cellpoint.png" width = "800" height = "400" div align=center />

### Quatities:

On point: ($x_p$,$y_p$), ($u_p$,$v_p$)

Zonal:  $V_z$, $\rho_z$, $m_z$, $e_z$, $p_z$

By the relation 
$$\frac{dm_z}{dt} =0$$
we have 
$$\rho_z(t) = m_z/V_z(t)$$

Also 
$$d(x_p)/dt = u_p, \quad d(y_p)/dt = v_p$$

### Point mass 

Point mass $m_p$ is used to solve point momentum $\mu_p = m_p \bf{u}_p$ and kinetic energy $K_p = m_p \frac{\bf{u}_p^2}{2}$

<img src="Ins.jpg" width = "200" height = "400" div align=center /> 

$$m_p = \sum_{z\in Z(p)} A_z^p \rho^p_z$$

$$\rho_z(t) = m_z/V_z(t)$$

In [4]:
function MassAndDens(mz,x,y)
    
    Ic, = size(mz)
    Ip, = size(Tc)
    Asbz = zeros(Float64,4)
    ρ = zeros(Float64,Ic)
    mp = zeros(Float64,Ip)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ic
        xz[1:4] = x[Tp[i,1:4]]
        yz[1:4] = y[Tp[i,1:4]]
        Asbz = A_z(xz,yz)
        Az = sum(Asbz)
        ρ[i] = mz[i]/Az
        
        for j = 1:4
            ip = Tp[i,j]
            mp[ip] = mp[ip] + Asbz[j]*ρ[i]
        end
    end
   # @show mp
    return ρ,mp   
end

MassAndDens (generic function with 1 method)

In [5]:
function density(mz,A)
    I = 
    for i in length(mz)
    ρ[i] = mz ./ (A[i,1] + A[i,2] +A[i,3] +A[i,4])
    return ρ
end

function mass_point(ρ,A)
    Ic, = size(mz)
    Ip, = size(Tc)
    mp = zeros(Float64,Ip)
    for j = 1:4
        for i in 1:Ic
            ip = Tp[i,j]
            mp[ip] += A[i,j]*ρ[i]
        end    
    end
    return mp
end

LoadError: syntax: incomplete: "function" at none:1 requires end


$$ A_z^1 = \frac{5A_{41}+5A_{12}+A_{23}+A_{34}}{12}$$
$$ A_z^2 = \frac{A_{41}+5A_{12}+5A_{23}+A_{34}}{12}$$
$$ A_z^3 = \frac{A_{41}+A_{12}+5A_{23}+5A_{34}}{12}$$
$$ A_z^4 = \frac{5A_{41}+A_{12}+A_{23}+5A_{34}}{12}$$

In [6]:
function area4(xz,yz) 
    A =zeros(Float64,4)
    xc= sum(xz)/4
    yc= sum(yz[i])/4
    
    A41 = area3(xz[4],xz[1],xc,yz[4],yz[1],yc)
    A12 = area3(xz[1],xz[2],xc,yz[1],yz[2],yc)
    A23 = area3(xz[2],xz[3],xc,yz[2],yz[3],yc)
    A34 = area3(xz[3],xz[4],xc,yz[3],yz[4],yc)

    A[1] = (5A41+5A12+A23+A34)/12
    A[2] = (A41+5A12+5A23+A34)/12
    A[3] = (A41+A12+5A23+5A34)/12
    A[4] = (5A41+A12+A23+5A34)/12
    
#    A[1] = (A41+A12+A23+A34)/4
#    A[2] = (A41+A12+A23+A34)/4
#    A[3] = (A41+A12+A23+A34)/4
#    A[4] = (A41+A12+A23+A34)/4
    
    return A
end

area4 (generic function with 1 method)

动量守恒方程：
\begin{equation}\label{eq:3}
  \begin{array}{l}
	\displaystyle \rho \frac{du}{dt}=\frac{\partial \sigma _{rr}}{\partial r}+ \frac{\partial s_{rz}}{\partial z} + \frac{\sigma_{rr}-\sigma_{\theta \theta}}{r}\\[0.3cm]
  \displaystyle \rho \frac{dv}{dt}=\frac{\partial \sigma _{zz}}{\partial z}+ \frac{\partial s_{rz}}{\partial r} + \frac{s_{rz}}{r}\\
  \end{array}
  \end{equation}

<img src="n12.png" width = "300" height = "300" div align=center />

$$\overrightarrow{n}_{12} = \frac{1}{L_{12}}(z_2- z_1, r_1-r_2)  $$

$$\overrightarrow{\tau}_{12} = \frac{1}{L_{12}}(r_2- r_1, z_2-z_1)  $$

$$ \mathbf{F_1}  =  \left(\frac{1}{2} ( -\sigma_{rr}(z_2-z_4)+ s_{rz}(r_2 -r_4)) + \frac{A}{4}\frac{(\sigma_{rr}-\sigma_{\theta\theta})}{r_c}, \frac{1}{2}(\sigma_{zz}(r_2-r_4) +s_{rz}(z_4 -z_2) ) + \frac{A}{4} \frac{s_{rz}}{r_c }\right)$$ 

In [7]:
function force_zone(x,y,u,v,ρ,p,σ) #(r,z)
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
    
    
    Fx[1] = 0.5(σ[1]*(y[4] - y[2]) + σ[3]*(x[2] - x[4])) 
    Fy[1] = 0.5(σ[3]*(y[4] - y[2]) + σ[2]*(x[2] - x[4]))
    
    Fx[2] = 0.5(σ[1]*(y[1] - y[3]) + σ[3]*(x[3] - x[1]))
    Fy[2] = 0.5(σ[3]*(y[1] - y[3]) + σ[2]*(x[3] - x[1]))
    
    Fx[3] = 0.5(σ[1]*(y[2] - y[4]) + σ[3]*(x[4] - x[2]))
    Fy[3] = 0.5(σ[3]*(y[2] - y[4]) + σ[2]*(x[4] - x[2]))
    
    Fx[4] = 0.5(σ[1]*(y[3] - y[1]) + σ[3]*(x[1] - x[3]))
    Fy[4] = 0.5(σ[3]*(y[3] - y[1]) + σ[2]*(x[1] - x[3]))
      
    ex,ey = hourglass_viscosity(x, y, u, v, ρ, p,σ) 

    Fx = Fx .+ ex
    Fy = Fy .+ ey
    
    return Fx,Fy
end           

force_zone (generic function with 1 method)

In [8]:
function force_zone(x,y,u,v,ρ,p,σ)
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
    A = area_quadrangle(x, y) 
    xc = sum(x)/4
    yc = sum(y)/4
    
    for i in 1:4
        i == 1 ? im = 4 : im = i-1
        i == 4 ? ip = 1 :  ip = i+1
        Fx[i] = 0.5(σ[1]*(y[im] - y[ip]) + σ[3]*(x[ip] - x[im])) +A/4*(2σ[1]+σ[2])/xc 
        Fy[i] = 0.5(σ[3]*(y[im] - y[ip]) + σ[2]*(x[ip] - x[im])) +A/4*(σ[3])/xc
    end
      
    ex,ey = hourglass_viscosity(x, y, u, v, ρ, p,σ) 

    Fx = Fx .+ ex
    Fy = Fy .+ ey
    
    return Fx,Fy
end           

force_zone (generic function with 1 method)

\begin{equation}
  \begin{aligned}
	&\displaystyle \rho \frac{de}{dt}= \mathbf{\sigma} \cdot \mathbf{\varepsilon}\\ 
	&=
	\sigma _{rr}\frac{\partial u}{\partial r}+\sigma_{zz}\frac{\partial v}{\partial z}+ \sigma_{\theta\theta} \frac{u}{r} + \sigma_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)
	\end{aligned}
	\end{equation}
    
\begin{equation}
  \begin{aligned}
	&\displaystyle M \frac{de}{dt}=  
\int_{\Omega}  r\sigma _{rr}\frac{\partial u}{\partial r}+r\sigma_{zz}\frac{\partial v}{\partial z}+ \sigma_{\theta \theta} u + r \sigma_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)dr dz
	\end{aligned}
	\end{equation}



\begin{equation} 
\int_{\Omega}  r\sigma _{rr}\frac{\partial u}{\partial r}  dr dz= \int _\Omega (\frac{\partial ru}{\partial r}  - u )dr dz 
	\end{equation}



# Infinite element method to solve the integration



\begin{equation}
  \begin{aligned}
\int_{\Omega}  \frac{\partial u}{\partial r} d\Omega&=  -\oint_{\partial \omega}u dz\\
  &= \frac{1}{2} \left((u_1-u_3)(z_2-z_4)+(u_2-u_4)(z_3-z_1)\right)
  \end{aligned}
\end{equation}

\begin{equation}
  \begin{aligned}
\int_{\Omega}  \frac{\partial u}{\partial z} d\Omega&= \oint_{\partial \omega}u dr\\
  &= -\frac{1}{2} \left((u_1-u_3)(r_2-r_4)+(u_2-u_4)(r_3-r_1)\right)
  \end{aligned}
\end{equation}

$$ \frac{\partial u}{\partial x}  \approx \frac{1}{2A}((u_1-u_3)(y_2-y_4)+(u_2-u_4)(-y_1+y_3))$$ 

$$ \frac{\partial v}{\partial y}  \approx -\frac{1}{2A}((v_1-v_3)(x_2-x_4)+(v_2-v_4)(-x_1+x_3))$$ 

对于节点
$$ \rho \frac{d \mathbf{u}}{dt} = - \nabla p$$

$$m_p \frac{d\bf{u}_p}{dt} = \sum_{p\in P(z)} \bf{F}_z^p$$

$$F_z^1 = \mathbf{F}_1 + c\mathbf{e}_1$$

本构方程：
\begin{equation}
  \begin{array}{l}
	\displaystyle \frac{ds_{rr}}{dt}=2\mu \left(\frac{\partial u}{\partial r}-\frac{1}{3}\nabla \cdot \vec{u} \right) +s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{zz}}{dt}=2\mu \left(\frac{\partial v}{\partial z}-\frac{1}{3}\nabla \cdot \vec{u} \right) -s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{rz}}{dt}=\mu \left(\frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right) - \frac{s_{rr}-s_{zz}}{2}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\
  \end{array}
  \end{equation}

Von Mises屈服条件：
\begin{equation}
  \frac{3}{2}(s_{rr}^2+s_{zz}^2+s_{\theta \theta}^2+2s_{xy}^2)\le (Y^0)^2
\end{equation}


In [9]:
function rhs(var::Var)
    μ = con1.μ
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    
    Ic, = size(Tp)
    Ip, = size(Tc)
    rhsu = zeros(Float64,Ip)
    rhsv = zeros(Float64,Ip) 
    rhse = zeros(Float64,Ic)
    rhss = zeros(Float64,Ic,3)
    σ = zeros(Float64,Ic,3)
  
    σ[:,1] = s[:,1] - p[:]
    σ[:,2] = s[:,2] - p[:]
    σ[:,3] = s[:,3] 
    
    rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)
    
    for i in 1:Ic
        for j =1:4
            rcell[j] = r[Tp[i,j]]
            zcell[j] = z[Tp[i,j]]
            ucell[j] = u[Tp[i,j]]
            vcell[j] = v[Tp[i,j]]
        end
        Az = sum(A_z(rcell,zcell))
        Fx,Fy = force_zone(rcell,zcell,ucell,vcell,ρ[i],p[i],σ[i,1:3])  
        ∂u_∂r,∂u_∂z = ∂uv(rcell,zcell, ucell)
        ∂v_∂r,∂v_∂z = ∂uv(rcell,zcell, vcell)
        rhse[i] = σ[i,1]*∂u_∂r + σ[i,2]*∂v_∂z+σ[i,3]*(∂u_∂z + ∂v_∂r)
        for j = 1:4
            ip = Tp[i,j]
            rhsu[ip] +=  Fx[j]
            rhsv[ip] +=  Fy[j]
        end  
        ∇u = ∂u_∂r + ∂v_∂z
    
        rhss[i,1] =  (2μ*(2/3*∂u_∂r - 1/3*∂v_∂z) + σ[i,3]*(∂u_∂z - ∂v_∂r)) ./Az
        rhss[i,2] =  (2μ*(2/3*∂v_∂z - 1/3*∂u_∂r) - σ[i,3]*(∂u_∂z - ∂v_∂r)) ./Az
        rhss[i,3] =   (μ*(∂u_∂z+∂v_∂r)  -   (σ[i,1] - σ[i,2])*(∂u_∂z - ∂v_∂r)) ./Az      
    end

    return rhse, rhsu,rhsv,rhss

end     

rhs (generic function with 1 method)

In [10]:
function output(f)
    io = open("data2.dat", "w+")
     writedlm(io,f,"  ") 
    close(io)
end

output (generic function with 1 method)

$$ \mathbf{S} = \mathbf{S} \times \text{min} \left(1,Y_0/\sqrt{\frac{3}{2}\mathbf{S}:\mathbf{S}}\right)$$

In [11]:
function yield_s!(s,con)
    Y0 = con.Y0
    I, = size(s)
    for i in 1:I
         sums = s[i,1]^2 +s[i,2]^2 +2*s[i,3]^2 +(s[i,1]+s[i,2])^2
        for j = 1:3
       
        s[i,j] = s[i,j] * min(1,Y0/√(3/2*sums))
            end 
    end
    return s
end

yield_s! (generic function with 1 method)

In [12]:
function predictor_corrector2nd(dt,var::Var,mz)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,r,z)
    e = p_to_e(ρ, p,con1)    
    p = artificial_viscosity!(r,z,u,v,ρ,p)
    ρ,p,s = bound_ghost_cell!(ρ,p,s)
    var = Var(r, z, u, v, ρ, p, s)
    
    rhse, rhsu,rhsv,rhss = rhs(var)
    rhsu,rhsv = bound_force(rhsu,rhsv,var)
    
    s = yield_s!(s,con1)
    r₀ =  r + dt*u
    z₀  = z + dt*v
    u₀  = u + dt*(rhsu ./ mp)
    v₀  = v + dt*(rhsv ./ mp)
    e₀  = e + dt*(rhse ./ mz) 
    s₀  = s + dt*rhss
  #  @show rhsu
    s₀ = yield_s!(s₀,con1)
    r₀,z₀,u₀,v₀ = bound_ghost_coordinate!(r₀,z₀,u₀,v₀)
    ρ,mp = MassAndDens(mz,r₀,z₀)
    p = e_to_p(ρ,e₀,con1)
    p = artificial_viscosity!(r₀,z₀,u₀,v₀,ρ,p) 
    ρ,p,s₀ = bound_ghost_cell!(ρ,p,s₀)
    var = Var(r₀, z₀, u₀, v₀, ρ, p, s₀)
   
    rhse₀,rhsu₀,rhsv₀,rhss₀ = rhs(var)
    rhsu₀,rhsv₀ = bound_force(rhsu₀,rhsv₀,var)
  #  @show s[:,1]
    r +=  0.5dt*(u + u₀)
    z +=  0.5dt*(v + v₀)
    u +=  0.5dt*(rhsu + rhsu₀) ./ mp
    v +=  0.5dt*(rhsv + rhsv₀) ./ mp
    e +=  0.5dt*(rhse + rhse₀) ./ mz
    s  += 0.5dt*(rhss + rhss₀)
    s = yield_s!(s,con1)
    r,z,u,v = bound_ghost_coordinate!(r,z,u,v)
    ρ,mp = MassAndDens(mz,r,z)
    p = e_to_p(ρ,e,con1)
    ρ,p,s = bound_ghost_cell!(ρ,p,s)
    var = Var(r,z,u,v,ρ,p,s)
   
    return var
end

predictor_corrector2nd (generic function with 1 method)

In [13]:
function fη(ρ,c::Const)
    η = ρ/c.ρ0
    fη=(η .-1.0) .* (η .-c.Γ0*(η .-1.0)/2.0) ./ (η .-c.s0*(η .-1)) .^2
end

function fηη(ρ,c::Const)
    η = ρ/c.ρ0
    fηη=(η .+(c.s0-c.Γ0) .* (η .-1)) ./(η .-c.s0*(η .- 1)) .^3
end

fηη (generic function with 1 method)

In [14]:
function euler1st(dt,var::Var,mz)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,r,z)
    e = p_to_e(ρ, p,con1)    
    p = artificial_viscosity!(r,z,u,v,ρ,p)
    ρ,p,s = bound_ghost_cell!(ρ,p,s)
    var = Var(r, z, u, v, ρ, p, s)
    
    rhse, rhsu,rhsv,rhss = rhs(var)
    rhsu,rhsv = bound_force(rhsu,rhsv,var)
    
 #   @show rhsv
    s = yield_s!(s,con1)
    r =  r + dt*u
    z = z + dt*v
    u  = u + dt*(rhsu ./ mp)
    v  = v + dt*(rhsv ./ mp)
    e  = e + dt*(rhse ./ mz) 
    s  = s + dt*rhss
  #  @show rhsu
    s = yield_s!(s,con1)
    r,z,u,v = bound_ghost_coordinate!(r,z,u,v)
    ρ,mp = MassAndDens(mz,r,z)
    p = e_to_p(ρ,e,con1)
    ρ,p,s = bound_ghost_cell!(ρ,p,s)
    var = Var(r,z,u,v,ρ,p,s)
   
    return var
end

euler1st (generic function with 1 method)

### Boundary force

If one edge of a cell is a free boundary, then $f$ on the points is zero. As we have add it in RHs(), we need to remove it by resolve it again. Different from RHs(), at every point of a cell the force is composited by two parts on different edges. This is distinguished by $k$ when $k = 1$ the boundary is on y direction, forces on 12 and 34 boundary must be subtracted. When $ k =2$ the boundary is on x direction, subtract forces on 41, 23. 

In [15]:
function bound_force(rhsu,rhsv,var)
    Ip, = size(Tc)
    Ib, = size(Tb)
   
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)
    ip1 = zeros(Int,4)
    σ = zeros(Float64,3)
    for i in 1:Ib
        if Tb[i,4] == 1 || Tb[i,4] == 3
            ic = Tb[i,1]        
            rcell[1:4] = r[Tp[ic,1:4]]
            zcell[1:4] = z[Tp[ic,1:4]]
            ucell[1:4] = u[Tp[ic,1:4]]
            vcell[1:4] = v[Tp[ic,1:4]]

            σ[1] = s[ic,1] - p[ic]
            σ[2] = s[ic,2] - p[ic]
            σ[3] = s[ic,3]

            ex,ey = hourglass_viscosity(rcell, zcell, ucell, vcell, ρ[ic]
                                        , p[ic],s[ic,:])
            i₁ = Tb[i,3]
            i₂ = i₁+1
            if i₁ == 4
                i₂ = 1
            end
            nx = zcell[i₂] -zcell[i₁]
            ny = rcell[i₁] -rcell[i₂]
            p0 = 1e-10
            for j in (i₁,i₂)

                ip = Tp[ic,j]
                    Fx = -σ[1]*nx/2 -σ[3]*ny/2+ex[j] #*nx^2/(nx^2+ny^2)  #+ex[j] 
                    Fy = -σ[3]*nx/2 -σ[2]*ny/2+ey[j] #*ny^2/(nx^2+ny^2)
                    Fx1 = (Fx*nx+Fy*ny)*nx/(nx^2+ny^2)
                    Fy1 = (Fx*nx+Fy*ny)*ny/(nx^2+ny^2)
                    Fx = Fx- Fx1
                    Fy = Fy - Fy1
                    Fx1 =  p0*nx/2 #+ex[j]*nx^2/(nx^2+ny^2) 
                    Fy1 =  p0*ny/2 #+ey[j]*ny^2/(nx^2+ny^2)

                    rhsu[ip] +=  -Fx1 # -Fx1
                    rhsv[ip] +=  -Fy1 #-Fy1

                if Tb[i,4] == 3 # boundary
                    uL = 2e-3
                    u[ip] = uL #*nx/(nx^2 +ny^2)
                end  
            end
        end
    end 

    return rhsu, rhsv
end  

bound_force (generic function with 1 method)

<img src="bound_corresponding.png" width = "400" height = "800" div align=center /> 

$$ Ig_1 = Tb[i,3] $$
$$Ib_1 = Ig_1 -1$$

In [16]:
function bound_ghost_coordinate!(r,z,u,v)
    Ip, = size(Tc)
    Ic, = size(Tp)
    Ib, = size(Tb)
   # r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
   
    rt= copy(r)
    zt= copy(z)
   # output(r)
    for i in 1:Ib
        if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            ig₁ = Tb[i,3]
            
           
            ig₁ == 4 ? ig₂ = 1 : ig₂ = ig₁+1
            ig₂ == 4 ? ig₃ = 1 : ig₃ = ig₂+1
            ig₃ == 4 ? ig₄ = 1 : ig₄ = ig₃+1

            ig₁ == 1 ? ib₁ = 4 : ib₁ = ig₁-1
            ib₁ == 1 ? ib₂ = 4 : ib₂ = ib₁-1
            ib₂ == 1 ? ib₃ = 4 : ib₃ = ib₂-1
            ib₃ == 1 ? ib₄ = 4 : ib₄ = ib₃-1

                r1 =    rt[Tp[ic2,ib₁]]
                r2 =    rt[Tp[ic2,ib₂]]
                z1 =    zt[Tp[ic2,ib₁]]
                z2 =    zt[Tp[ic2,ib₂]]

                r3=     rt[Tp[ic2,ib₃]]
                z3=     zt[Tp[ic2,ib₃]]
                r4=     rt[Tp[ic2,ib₄]]
                z4=     zt[Tp[ic2,ib₄]]
                
                nx = zt[Tp[ic2,ib₁]] -zt[Tp[ic2,ib₂]]
                ny = rt[Tp[ic2,ib₂]] -rt[Tp[ic2,ib₁]]
                for j in (ib₁,ib₂)
                    ip = Tp[ic2,j]
                    u[ip] = u[ip]*ny^2/(nx^2 +ny^2)
                    v[ip] = v[ip]*nx^2/(nx^2 +ny^2)
                end

                r[Tp[ic,ig₃]],z[Tp[ic,ig₃]] = reflect(r1,z1,r2,z2,r3,z3)
                r[Tp[ic,ig₄]],z[Tp[ic,ig₄]] = reflect(r1,z1,r2,z2,r4,z4)
                u[Tp[ic,ig₃]],v[Tp[ic,ig₃]] = u[Tp[ic2,ib₃]],v[Tp[ic2,ib₃]]
                u[Tp[ic,ig₄]],v[Tp[ic,ig₄]] = u[Tp[ic2,ib₄]],v[Tp[ic2,ib₄]]
            
       
          end 
    end
    return r,z,u,v
end  

bound_ghost_coordinate! (generic function with 1 method)

In [17]:
function bound_ghost_cell!(ρ,p,s)
    Ip, = size(Tc)
    Ic, = size(Tp)
    Ib, = size(Tb)    
    for i in 1:Ib
        if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            ρ[ic] = ρ[ic2]
            p[ic] = p[ic2]
            s[ic,1:2] = s[ic2,1:2]
            s[ic,3] = -s[ic2,3]
          end 
    end
    return ρ,p,s
end  

bound_ghost_cell! (generic function with 1 method)

In [18]:
exp1[r4]

UndefVarError: UndefVarError: exp1 not defined

## Ghost cell coordinate of a reflect boundary

<img src="reflect.png" width = "200" height = "400" div align=center /> 

$$ k_1 k_2 = -1$$
$$ d_3 = -d_4 $$

$$(z_2-z_1)(z_3-z_4)+(r_3-r_4)(r_2-r_1) = 0$$
$$(z_2-z_1)(r_3-r_1)-(z_3-z_1)(r_2-r_1) = -(z_2-z_1)(r_4-r_1)+(z_4-z_1)(r_2-r_1)$$

In [19]:
using SymPy
@vars z1 z2 z3 z4 r1 r2 r3 r4
exp1 = solve([(z2-z1)*(z3-z4)+(r2-r1)*(r3-r4),(z2-z1)*(r3-r1)-(z3-z1)*(r2-r1)+(z2-z1)*(r4-r1)-(z4-z1)*(r2-r1)],[r4,z4])
@show exp1[r4],exp1[z4]

ArgumentError: ArgumentError: Package SymPy not found in current path:
- Run `import Pkg; Pkg.add("SymPy")` to install the SymPy package.


In [20]:
function reflect(r1,z1,r2,z2,r3,z3)
    tmp1=(r1*r3-r2*r3+z1*z3-z2*z3) 
    tmp2 = (2r1*z2-r1*z3-2r2*z1+r2*z3+r3*z1-r3*z2)
    tmp3 = ((r1-r2)^2+(z1-z2)^2)
    
    r4 = ((r1-r2)*tmp1 - (z1-z2)*tmp2)/tmp3
    z4 = ((r1-r2)*tmp2 + (z1-z2)*tmp1)/tmp3
    
    return r4,z4
end

reflect (generic function with 1 method)

In [21]:
reflect(1,1,1,0,0.9,0)

(1.1, 0.0)

### EOS 
$$ e = \frac{p}{(\gamma-1)\rho}$$

In [22]:
function p_to_e(ρ,p,con)
    c=con
    ei = (p .- c.ρ0*c.a0^2*fη(ρ,c))/(c.ρ0*c.Γ0)
    return ei
end
function e_to_p(ρ,ei,con::Const)
    c=con
    p = c.ρ0*c.Γ0*ei .+ c.ρ0*c.a0^2*fη(ρ,c)
    return p
end

e_to_p (generic function with 1 method)

In [23]:
function CFL(SF,var)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    Ic, = size(ρ)
    cflmin = 1.e9
      cfl = 0 
    for i in 1:Ic
        cfl =0
        u2max = 0
        u2=0.0
        for j = 1:4
            ip = Tp[i,j]
            u2 = u[ip]^2+v[ip]^2
            if u2 > u2max ; u2max = u2; end
        end
        
        dlmin = (r[Tp[i,4]] - r[Tp[i,1]])^2+(z[Tp[i,4]]-z[Tp[i,1]])^2
       
        for j=1:3
            dl= (r[Tp[i,j+1]] - r[Tp[i,j]])^2+(z[Tp[i,j+1]]-z[Tp[i,j]])^2
            if dl < dlmin; dlmin=dl; end
        end
        c = sound([ρ[i],u[i],p[i],min(s[i,1],s[i,2])],con1)
        cfl = √(dlmin)/(√(u2max)+c)
        if cfl < cflmin; cflmin = cfl; end
    end
 #   @show cflmin
    dt = cflmin*SF
    return dt
end 
        

CFL (generic function with 1 method)

In [24]:
function sound(uo::Array{Float64,1},con::Const,EoP::Int=1)
    a0,ρ0,Γ0,Y0,μ  = con.a0,con.ρ0,con.Γ0,con.Y0,con.μ
    ρ,uu,p,sxx   = uo[1:4]
    a2  = a0^2*fηη(ρ,con)+p/ρ^2*ρ0*Γ0
    if EoP == 2
        c=sqrt(a2-ρ0/ρ^2*Γ0*sxx)
        return c
    else
       #  @show a2,ρ0/ρ^2,4.0/3*μ/ρ
        c=sqrt(a2-ρ0/ρ^2*Γ0*sxx+4.0/3*μ/ρ)
       
        return c
    end
end

sound (generic function with 2 methods)

In [25]:
function TimeSolve(var,mz)
    tt= problem.tt
    sf =problem.sf
    t= 0.0
    t₁ = tt/100
    t₂ = t₁
    kinetic_init,energy_init = total_kinetic(var,mz)
    kinetic = kinetic_init
   # while kinetic > 1e-4*kinetic_init && t<tt
    while t<tt
    
#for i in 1:50
        dt=CFL(sf,var)
   #     @show dt
        if t+dt>tt
           dt = tt-t
        end
        
        var = predictor_corrector2nd(dt,var,mz)
       # var = euler1st(dt,var,mz)
        kinetic,energy = total_kinetic(var,mz)
         t += dt
        if t > t₁
            println(t)
            t₁ += t₂
            @show kinetic/kinetic_init,(energy+kinetic)/(kinetic_init+energy_init)
        end
        
    end
    return var
end

TimeSolve (generic function with 1 method)

In [26]:
function total_kinetic(var,mz)
    r,z,u,v,ρ,p = var.r,var.z,var.u,var.v,var.ρ,var.p
    Ip, = size(Tc)
    Ic, = size(Tp)
    Ib, = size(Tb)
    k = zeros(Float64,Ip)
    
    kinetic = 0
    energy = 0
     ρ1,mp = MassAndDens(mz, r, z)
    
    for i in 1:Ip
        k[i]= mp[i]*0.5*(u[i]^2+v[i]^2)
    end
    
    
    for i in 1:Ic
        energy += mz[i]*p_to_e(ρ[i], p[i], con1)
    end
    
    for i in 1:Ib
        if Tb[i,4] == 2
            ic= Tb[i,1]
            energy -= mz[ic]*p_to_e(ρ[ic], p[ic], con1)
            
            ig₁ = Tb[i,3]
            ig₁ == 4 ? ig₂ = 1 : ig₂ = ig₁+1
            k[Tp[ic,ig₁]] = 0
            k[Tp[ic,ig₂]] = 0
            end
    end
    kinetic = sum(k)
    return kinetic,energy
end


total_kinetic (generic function with 1 method)

## Artificial Viscosity 

Use the strain rate to construct the artificial viscosity 
$$\frac{ds}{dt} = \frac{\partial u}{\partial x}  + \frac{\partial v}{\partial y} $$


Then the artificial viscosity can be written as
$$ q_w = \left\{ \begin{align}
\alpha L \rho \left( \frac{ds}{dt} \right)^2 +  \beta L^2 \rho c \left|\frac{ds}{dt}\right|, \quad \text{if}  \quad \frac{ds}{dt}< 0,\\
0, \quad \text{if} \quad \frac{ds}{dt}\ge 0. \\
\end{align}
\right.
$$
where $\alpha = 0.6$, $\beta = 2.0 $ and $c$ is the sonic speed.

$L$ is the reference length, and constructed as
$$L = A/L_{\text{max}} $$

$$L_{\text{max}} = \text{max}(L_{13},L_{24})$$


In [27]:
function artificial_viscosity_c!(x,y,u,v,ρ,p)
    xc = sum(x[i] for i in 1:4)/4
    yc = sum(y[i] for i in 1:4)/4
    
    ∂u_∂x,tmp = ∂uv(x,y,u)
    tmp,∂v_∂y = ∂uv(x,y,v)
    c = sound([ρ,0.0,p,0.0],con1)
    Lmax = max(√((x[3] - x[1])^2+(y[3] - y[1])^2) ,√((x[2] - x[4])^2+(y[2] - y[4])^2) )  
    A = area_quadrangle(x, y)
    L = A/Lmax 

    if ∂u_∂x + ∂v_∂y> 0
        q = 0
    else
        q = 0.6ρ*L*c*(abs(∂u_∂x+∂v_∂y))/A +2.0L^2*ρ*((∂u_∂x+∂v_∂y)^2)/A^2 
    end
    return p+q
end

artificial_viscosity_c! (generic function with 1 method)

$$b_2 \rho Lc \frac{\partial u}{\partial x} - b_{12}\rho L^2 \left(\frac{\partial u}{\partial x}\right)^2$$

In [28]:
function artificial_viscosity!(r,z,u,v,ρ,p)
    I, = size(Tp)
    rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)
   
    for i in 1:I
        for j in 1:4
            ip = Tp[i,j]
            rcell[j] = r[ip]
            zcell[j] = z[ip]
            ucell[j] = u[ip]
            vcell[j] = v[ip]
        end
        p[i] = artificial_viscosity_c!(rcell,zcell,ucell,vcell,ρ[i],p[i])
    end
    
        return p
        
    end       

artificial_viscosity! (generic function with 1 method)

In [29]:
function l_viscosity(x,y,α)
    xc = sum(x)/4; yc = sum(y)/4
    d = zeros(Float64,4)
    for i in 1:4
    d[i] = distance(x[i],y[i],xc,yc,α)
    end
    A = A_z(x, y)
    L = 2sum(A)/sum(d)
    return L
end

l_viscosity (generic function with 1 method)

#### distance from  Point to line 
Line: $$ (xc -cos \alpha )(x-xc) = (yc-sin \alpha)(y-yc)$$
point: $x,y$

In [30]:
function distance(x,y,xc,yc,α)
    d = abs((xc - cosd(α)) * (x .-xc)- (yc-sind(α))*(y .- yc))/√((xc-cosd(α))^2+(yc-sind(α))^2)
    return d
end

distance (generic function with 1 method)

### 沙漏粘性 Hourglass

$$ \text{hg}_x = u_1 -u_2 + u_3 - u_4$$ 
$$ \text{hg}_x = v_1 -v_2 + v_3 - v_4$$ 

$$ e_{1x} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{1y} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

$$ e_{2x} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{2y} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

$$ e_{3x} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{3y} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

$$ e_{4x} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{4y} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

In [31]:
function hourglass_viscosity(x,y,u,v,ρ,p,s)
    hgx = u[1] - u[2] + u[3] -u[4]
    hgy = v[1] - v[2] + v[3] -v[4]
    
    ex =zeros(Float64,4)
    ey =zeros(Float64,4)
    A = area_quadrangle(x,y)
    c = sound([ρ,0.0,p,min(s[1],s[2])], con1)
 #   @show c
    q = 0.1 # coefficient of hourglass viscosity 0.01 - 0.5 ?
    ex[1] = -0.25q*ρ*c*√(A)*hgx
    ey[1] = -0.25q*ρ*c*√(A)*hgy
    
    ex[2] = -ex[1]
    ey[2] = -ey[1]
    
    ex[3] = ex[1]
    ey[3] = ey[1]
    
    ex[4] = ex[2]
    ey[4] = ey[2]
    
  #ex .= 0
  #ey .= 0
 #   @show ex,ey
    return ex, ey
end

hourglass_viscosity (generic function with 1 method)

In [32]:
function area_quadrangle(x::Array{Float64,1},y::Array{Float64,1}) 
   
    xc= sum(x[i] for i in 1:4)/4
    yc= sum(y[i] for i in 1:4)/4
    
    A41 = Area3(x[4],x[1],xc,y[4],y[1],yc)
    A12 = Area3(x[1],x[2],xc,y[1],y[2],yc)
    A23 = Area3(x[2],x[3],xc,y[2],y[3],yc)
    A34 = Area3(x[3],x[4],xc,y[3],y[4],yc)
    
    A = A12 + A23 +A34 +A41
    return A
end

area_quadrangle (generic function with 1 method)

In [33]:
function Area3(x1,x2,x3,y1,y2,y3)
    return abs((x1*y2+y1*x3+x2*y3-x1*y3-y1*x2-y2*x3)/2)
end

Area3 (generic function with 1 method)

###  $\frac{\partial (u,v)}{\partial (x,y)}$

we use the diffences between 1 and 3 points and 2 and 4 points to solve the partial differences approximately, as
$$ \frac{\partial u}{\partial x}  \approx \frac{1}{2A}((u_1-u_3)(y_2-y_4)+(u_2-u_4)(-y_1+y_3))$$ 

$$ \frac{\partial v}{\partial y}  \approx -\frac{1}{2A}((v_1-v_3)(x_2-x_4)+(v_2-v_4)(-x_1+x_3))$$ 
Similar process to $\frac{\partial u}{\partial y}$,$\frac{\partial v}{\partial x}$ and $\frac{\partial v}{\partial y}$

In [34]:
function ∂uv(x,y,u)
    ∂u_∂x = 0.5(u[1]-u[3])*(y[2]-y[4]) + 0.5(u[2]-u[4])*(y[3]-y[1])
    ∂u_∂y = -0.5(u[1]-u[3])*(x[2]-x[4]) - 0.5(u[2]-u[4])*(x[3]-x[1])
    return ∂u_∂x,∂u_∂y
end

∂uv (generic function with 1 method)

## Output function

In [35]:
function cell_coordinate(x,y)
    Ip, = size(Tc)
    Ic, = size(Tp)
    xc = zeros(Float64,Ic)
    yc = zeros(Float64,Ic)
    
    for i in 1:Ic
        for j = 1:4
            ip = Tp[i,j] 
            xc[i] += x[ip]/4
            yc[i] += y[ip]/4
        end
        
    end
    
    return xc,yc
end

cell_coordinate (generic function with 1 method)

In [36]:
function outputline(Ix,Iy,var,mz)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    I, = size(ρ)
    
    uline =  zeros(Float64,Ix)
    rline =  zeros(Float64,Ix)
    zline =  zeros(Float64,Ix)
    ρline =  zeros(Float64,Ix)        
    pline =  zeros(Float64,Ix)  
    sline = zeros(Float64,Ix,3)
    for i in 1:Ix
        j = Int(floor(Iy/2))
            ip = i+(j-1 ) *(Ix+1)
            ic =i+(j-1)*Ix
            uline[i] = u[ip]
            rline[i]  = r[ip]
            ρline[i]  = ρ[ic]
            pline[i] = p[ic]
           # σline[i,:] .= σ[ic,:]
            sline[i,:] = s[ic,:]
    end
    return rline,uline,ρline,pline,sline
end

outputline (generic function with 1 method)

In [37]:
function Output_cell(Ix,Iy,var,mz)
    x,y,u,v,ρ,p,s= var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data.dat", "w+")
    A = zeros(Float64,Ip,10)
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"u\" \"v\" \"rho\" \"p\" \"srr\" \"szz\" \"szr\" \"s:s\"  ZONE T=\"Zone 1\" 
I=$Ix,J=$Iy,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "

    write(io,text) 
    for i in 1:Ic
        xc = 0
        yc =0
        uc =0
        vc =0
        for j = 1:4
            ip = Tp[i,j] 
                xc += x[ip]/4
                yc += y[ip]/4
                uc += u[ip]/4
                vc += v[ip]/4
        end
        
        A[i,1] = xc
        A[i,2] = yc
        A[i,3] = uc
        A[i,4] = vc
        A[i,5] = ρ[i]        
        A[i,6] = p[i]      
        A[i,7:9] .= s[i,1:3] 
        A[i,10] = (s[i,1]^2+s[i,2]^2 +2s[i,3]^2+(s[i,1]+s[i,2])^2)
    end
      writedlm(io, A, "  ")
    close(io)
end

Output_cell (generic function with 1 method)

In [51]:
function Output_point(Ix,Iy,var,mz)
    x,y,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    Ip, = size(Tc)
    Ic, = size(Tp)
    
    io = open("data.dat", "w+")
    #s = zeros(Float64,Ip,3)
    A = zeros(Float64,Ip,9)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"u\" \"v\" \"rho\" \"p\" \"srr\" \"szz\" \"szr\"   ZONE T=\"Zone 1\" 
I=$(Ix+1),J=$(Iy+1),K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)
    
       ρp = zeros(Float64,Ip)
       pp = zeros(Float64,Ip)
       np = zeros(Int,Ip)
       sp = zeros(Float64,Ip,3)
    
    for i in 1:Ic
        
        xc = 0
        yc =0
        uc =0
        vc =0
        for j = 1:4
            ip = Tp[i,j] 
            ρp[ip] += ρ[i]
            pp[ip] += p[i]
            sp[ip,:] += s[i,:]
            np[ip] += 1   
        end
    end
        ρp = ρp ./ np
        pp = pp ./ np
        sp = sp ./ np
    
        A[:,1] = x
        A[:,2] = y
        A[:,3] = u
        A[:,4] = v
        A[:,5] = ρp
        A[:,6] = pp
        A[:,7:9] .= sp[:,1:3]
    
      writedlm(io, A, "  ")
    close(io)
end

Output_point (generic function with 1 method)

## Cases

In [39]:
 function Piston()

    global problem = prb(1.5e2,0.1)
    # global problem = prb(3e-5,0.3)
     
    I = 200
    J = 5
    dx = 100/I
    dy = 10/J
    @show typeof(dx)
    I1= I+1
    J1= J+2
    global   con1 = Const(9e-5,8.930,2.0,4.5e-1,0.3940,1.49)
  #  global   con1 = Const(0.0,8930,2.0,4.5e10,3940.0,1.49)
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1-2,4) #边界 
    
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    

# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 4
            Tb[ib,4] = 3
        end
    
      for i = 1:I1-1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 2
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 2
                end
            end
        end
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 4
             Tb[ib,4] = 2
         end
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            x[ip] = (i-1)*dx
            y[ip] = (j-2)*dy
            u[ip] = 0
            v[ip] = 0.0
        end
    end
    
      
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y = bound_ghost_coordinate!(x,y)
    x,y = bound_ghost_coordinate!(x,y)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A)
                p[ic] = 1.e-7
                ρ[ic] = 8.930
                mz[ic] = ρ[ic] * Az[ic]
                s[ic,:] .= 0       
        end
    end 
    return var,mz,I1,J1
end

Piston (generic function with 1 method)

In [ ]:
var,mz,I,J=Piston()
#Output1
var= TimeSolve(var,mz)

In [ ]:
push!(LOAD_PATH,"/home/bfly/workspace/Juliastudy/Src")

In [ ]:
uo,u,x,inter = Lag1DNum.Piston()
uo,x  = Lag1DNum.TimeSolve(uo, x,inter)

In [ ]:
rcell, zcell = cell_coordinate(var.r,var.z)

In [52]:
using DelimitedFiles
Output_point(I,J,var,mz)

In [ ]:
 function Pistonz()

    global problem = prb(1.5e-4,0.3)
    # global problem = prb(3e-5,0.3)
     
    I = 10
    J = 100
    dx = 0.1/I
    dy = 1/J
    
    global   con1 = Const(9e7,8930,2.0,4.5e10,3940.0,1.49)
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, I*2+J*2,3) #边界 
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:I 
        for j in 1:J
            ic = i+(j-1)*I
            Tp[ic,1] = i + (j-1)*(I+1)
            Tp[ic,2] = i+1 +(j-1)*(I+1)
            Tp[ic,3]=i+1+j*(I+1)
            Tp[ic,4]=i+j*(I+1) 
        end
    end
    
    for i = 1:I+1
        for j = 1:J+1
            ip = i+(j-1)*(I+1)
            Tc[ip,1] = i +(j-1)*I #(i,j)
            Tc[ip,2] = i-1+(j-1)*I #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I   #(i,j-1)
            end
        end
    

# Bound 
  ib = 0
    for i = 1:I+1
        for j = 1:J+1
            if i == 1 || i== I+1 ||  j == 1 || j == J+1
                ib += 1
                ic =i+(j-1)*I
                #ic = Tc[ip]
                Tb[ib,1] = ic
                if i==1 
                    Tb[ib,2] = 4 # 1->4 
                    Tb[ib,3] = 2 # type 1:free 2:
                end   
                 if  i==I+1 
                    Tb[ib,2] = 2 #bound type 1:free 2: wall ..  Tb[:,2] y direction
                    Tb[ib,3] = 2
                end
            
                if j==1 
                    Tb[ib,2] = 1
                    Tb[ib,3] = 1
                end
                if j==J+1
                    Tb[ib,2] = 3 
                    Tb[ib,3] = 1
                end
            end
        end
    end

    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    σ = zeros(Float64,Ic,3)
    
    for i in 1:I+1
        for j = 1:J+1
            ip =i+(j-1)*(I+1)
            x[ip] = (i-1)*dx
            y[ip] = (j-1)*dy
            u[ip] = 0
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I
        for j in 1:J
            ic = i+(j-1)*I
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A) #[1]+A[2]+A[3]+A[4]
          #  if i>=I/2
               
                p[ic] = 1.e5
                ρ[ic] = 8930
                mz[ic] = ρ[ic] * Az[ic]
                σ[ic,1] = -p[ic]
                σ[ic,2] = -p[ic]
                σ[ic,3]= 0 
#             else
#                 mz[ic] = 1.0 * Az[ic]
#                 p[ic] = 1.0
#                 ρ[ic] = 1.0
#             end
        end
    end
    var = Var(x,y,u,v,ρ,p,σ)
    return var,mz,I,J
end

In [ ]:
var,mz,I,J=Pistonz()
#Output1
var = TimeSolve(var,mz)

In [ ]:
rcell, zcell = cell_coordinate(var.r,var.z)

In [ ]:
plot3D(var.r,var.z,var.v)
#triplot()

In [ ]:
tricontourf(var.r,var.z,var.v)

In [ ]:
? Const

## 单位变换

$$ 1kg/m^3 = 10^{-3} g/cm^3$$
$$1m/s = 10^{-4} cm/\mu s$$
$$ 1Pa = 1N/m^2 = 1kg/(m\cdot s^2) = (1000g)/(100cm \cdot 10^{12} \mu s^2) = 10^{-11} g/(cm \cdot s^2)$$ 

In [108]:
 function Collapse()

    global problem = prb(1.3e2,0.1)
  #  global problem = prb(50,0.3)
    # global problem = prb(5e-5,0.3)
    
    I = 20
    J = 16
    dr = 2.0/J
    dθ = 90/I
    
    I1 = I +2
    J1 = J
    
    global   con1 = Const(3.3e-3,1.845,2.0,1.519,1.287,1.124)
    
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
     Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1,4) #边界 
    
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    

# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 2
            Tb[ib,4] = 2
        end
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 4
             Tb[ib,4] = 2
         end
      for i = 1:I1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 1
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 1
                end
            end
        end
        
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            r₁ = (j-1)*dr+8
            θ = 90-(i-2)*dθ
            v₀ = 4.171e-2
            x[ip] = cosd(θ)*r₁
            y[ip] = sind(θ)*r₁
            u[ip] = -v₀*8/r₁ *cosd(θ)
            v[ip] = -v₀*8/r₁ *sind(θ)
        end
    end
      
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y,u,v = bound_ghost_coordinate!(x,y,u,v)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A)
                p[ic] = 1.e-5
                ρ[ic] = 1.845
                mz[ic] = ρ[ic] * Az[ic]
                s[ic,:] .= 0       
        end
    end 
    return var,mz,I1,J1
end

Collapse (generic function with 1 method)

In [109]:
var,mz,I,J = Collapse()
var = TimeSolve(var,mz)

1.3039221439716024
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9849354785644913, 0.9999978109878737)
2.6007636210726086
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9690472983094149, 0.9999942588223742)
3.905579672660331
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9529723491849652, 0.9999911870264317)
5.201318924684604
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9370208365989564, 0.999988333576505)
6.5032973483457095
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.921072696433555, 0.999984755310165)
7.803109765843356
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9052310752699532, 0.9999820432773296)
9.101489076952236
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.8894966936493964, 0.9999785897242837)
10.405463021359367
(kinetic / kinetic_ini

79.30773755038521
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.18227979989720172, 0.9998250923745775)
80.60705846510358
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.1732724735999379, 0.9998233702106889)
81.90014351605589
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.16450797044788082, 0.9998217186511906)
83.2054098782492
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.15586541851368474, 0.9998201177663498)
84.50428597969668
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.14747180961610015, 0.9998185942631286)
85.80596614058676
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.13926953460166186, 0.9998171325293809)
87.10106578192509
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.1313192953528227, 0.9998157437164643)
88.40815856336569
(kinetic / kinetic_

Var([-0.392023, 0.0, 0.392023, 0.78163, 1.16642, 1.54401, 1.91209, 2.26838, 2.61068, 2.93689  …  6.31723, 6.65787, 6.95745, 7.21414, 7.42635, 7.59278, 7.71239, 7.78446, 7.80853, 7.78446], [4.98113, 4.99653, 4.98113, 4.93501, 4.85848, 4.75198, 4.61619, 4.45194, 4.26024, 4.04228  …  4.58974, 4.07994, 3.545, 2.98819, 2.41297, 1.82286, 1.22152, 0.61265, 0.0, -0.61265], [2.55221e-5, 0.0, 2.55221e-5, 5.08867e-5, 7.59375e-5, 0.00010052, 0.000124483, 0.000147678, 0.000169962, 0.000191198  …  0.000193487, 0.000203916, 0.000213084, 0.000220936, 0.000227425, 0.000232514, 0.000236171, 0.000238374, 0.00023911, 0.000238374], [0.00032421, 0.000325212, 0.00032421, 0.00032121, 0.00031623, 0.000309303, 0.000300471, 0.000289787, 0.000277318, 0.000263138  …  0.00014056, 0.000124939, 0.000108549, 9.14928e-5, 7.3876e-5, 5.58066e-5, 3.73955e-5, 1.87552e-5, -0.0, 1.87552e-5], [1.84401, 1.84401, 1.84401, 1.84401, 1.84401, 1.84401, 1.84401, 1.84401, 1.84401, 1.84401  …  1.84498, 1.84498, 1.84498, 1.84498, 1.844

In [ ]:
var,mz,I,J = Collapse()
var = TimeSolve(var,mz)
using DelimitedFiles
(I,J,var,mz)

In [110]:
using DelimitedFiles
Output_cell(I,J,var,mz)

In [85]:
tmp =0.0
for i in 1:21
   tmp += √(var.r[i]^2 + var.z[i]^2)
end
@show tmp/21

tmp / 21 = 4.99603340740572


4.99603340740572

In [86]:
tmax =0.0
for i in 1:21
    tmp2 = √(var.r[i]^2 + var.z[i]^2)
    if abs(tmp2 - tmp/21) >tmax
        tmax = abs(tmp2-tmp/21)
    end
end
@show tmax

tmax = 1.3224976669334865e-12


1.3224976669334865e-12

In [ ]:
 rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)

Ic, = size(Tp) 
for i in 1:Ic
        for j =1:4
            rcell[j] = var.r[Tp[i,j]]
            zcell[j] = var.z[Tp[i,j]]
            ucell[j] = var.u[Tp[i,j]]
            vcell[j] = var.v[Tp[i,j]]
        end
        Az = sum(A_z(rcell,zcell))
@show Az
end

In [104]:
 function Impact()

    global problem = prb(5000,0.1)
    # global problem = prb(5e-5,0.3)
    
    I = 100
    J = 20
    dx = 500/I
    dy = 100/J
    
    global   con1 = Const(3e-3,2.785,2.0,2.7e-1,0.5328,1.338)
    
    I1 = I +1
    J1 = J
    
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1,4) #边界 
    
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    
# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 2
            Tb[ib,4] = 2
        end
    
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 2
             Tb[ib,4] = 1
         end
      for i = 1:I1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 1
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 1
                end
            end
        end
        
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            x[ip] = (i-2)*dx
            y[ip] = (j-1)*dy
            u[ip] = -1.50e-2
            v[ip] = 0.0
        end
    end
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y,u,v = bound_ghost_coordinate!(x,y,u,v)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A)
                 p[ic] = 1e-10
                ρ[ic] = 2.785
                mz[ic] = ρ[ic] * Az[ic]
                s[ic,:] .= 0
        end
    end 
 return var,mz,I1,J1
end

Impact (generic function with 1 method)

In [114]:
var,mz,I,J = Impact()
var = TimeSolve(var,mz)

50.32973643796638
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9485466456740563, 0.9974313331880645)
100.06664913224165
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.931579003653482, 0.9974270961807475)
150.1631269195772
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9251480787680706, 0.9974240353682438)
200.12920291957965
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.9060005044677821, 0.9974301350705642)
250.19060056165284
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.8820844491921004, 0.9974338689446197)
300.1304317580426
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.8678158960866412, 0.9974327903482122)
350.0425854941553
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.8557196314628925, 0.9974292939487116)
400.21653046610834
(kinetic / kinetic_i

3050.1153958320524
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.16416609280789876, 0.9972279989114494)
3100.084024450892
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.15640342629848164, 0.9972191085569612)
3150.0529183932676
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.14875620575406176, 0.9972084056663604)
3200.0816444363845
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.1414710664976176, 0.9971972799208684)
3250.013447177605
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.134286931425188, 0.9971877982254709)
3300.11468004044
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.12721981032689514, 0.9971791018922248)
3350.024560238917
(kinetic / kinetic_init, (energy + kinetic) / (kinetic_init + energy_init)) = (0.12047025262422795, 0.9971694396236495)
3400.047676969782
(kinetic / kineti

Var([-2.43129, 0.0, 2.43129, 4.74098, 7.00225, 9.2604, 11.5163, 13.7644, 16.0166, 18.2717  …  437.19, 439.689, 442.188, 444.687, 447.186, 449.686, 452.185, 454.685, 457.185, 459.685], [-37.7219, -38.0063, -37.7219, -37.1471, -36.5641, -35.9754, -35.385, -34.8011, -34.2181, -33.6271  …  99.9999, 99.9989, 99.9981, 99.9977, 99.9974, 99.9974, 99.9974, 99.9974, 99.9974, 99.9975], [-5.53997e-6, -0.0, -5.53997e-6, 4.38559e-7, -1.46352e-6, -7.77751e-6, -1.94089e-6, 7.49829e-6, 1.61537e-5, 1.89513e-5  …  -0.00035215, -0.000362538, -0.000370775, -0.000376769, -0.000381197, -0.000383567, -0.000383277, -0.000380116, -0.000377754, -0.000377492], [-8.49738e-5, -8.09077e-5, -8.49738e-5, -9.13681e-5, -9.1891e-5, -9.43001e-5, -0.000101233, -0.000109222, -0.000125303, -0.000143465  …  -5.72282e-5, -7.05329e-5, -7.93128e-5, -8.47208e-5, -8.79398e-5, -8.78117e-5, -8.57841e-5, -8.57193e-5, -8.81551e-5, -9.21397e-5], [2.77708, 2.77708, 2.77714, 2.77682, 2.77666, 2.77658, 2.77656, 2.77655, 2.77654, 2.77647  

In [112]:
using DelimitedFiles
Output_point(I,J,var,mz)

In [11]:
using PyCall

## 基函数变换法
$$\phi_i =\frac{1}{4} (1+\xi_i\xi)(1+\eta_i\eta) \quad i = (1,2,3,4)$$

$$ (\xi_1,\eta_1) = (-1,-1), \quad (\xi_2,\eta_2) = (1,-1), \quad (\xi_3,\eta_3) = (1,1),\quad (\xi_4,\eta_4) = (-1,1)$$
$$ r = \sum r_i \phi_i ,\quad  z = \sum z_i \phi_i ,\quad u = \sum u_i \phi_i,\quad  v = \sum v_i \phi_i $$


$$ J =\frac{\partial r}{\partial \xi} \frac{\partial z}{\partial \eta} - \frac{\partial r}{\partial \eta}\frac{\partial z}{\partial \xi}$$

$$\int_\Omega d\Omega = \int _{-1}^1 \int_{-1}^1 J d\xi d\eta $$


$$ J = \frac{1}{16}\sum_1^4 \xi_i r_i (1+\eta_i\eta)\sum_1^4 \eta_i z_i (1+ \xi_i \xi) - \frac{1}{16}\sum_1^4 \eta_i r_i (1+\xi_i\xi) \sum_1^4 \xi_i z_i (1+ \eta_i \eta) $$

In [48]:
@vars r₁ r₂ r₃ r₄ η ξ  z₁ z₂ z₃ z₄ r_ξ z_η r_η z_ξ
r =[r₁,r₂,r₃,r₄]
z = [z₁,z₂,z₃,z₄]
ξn = [-1,1,1,-1]
ηn = [-1,-1,1,1]

r_ξ=0; z_η=0; r_η =0; z_ξ =0
for i in 1:4
    r_ξ += ξn[i]*r[i]*(1+ηn[i]*η)
    z_η += ηn[i]*z[i]*(1+ξn[i]*ξ)
    r_η += ηn[i]*r[i]*(1+ξn[i]*ξ)
    z_ξ += ξn[i]*z[i]*(1+ηn[i]*η)
end
y = r_ξ*z_η -r_η*z_ξ
y = integrate(y,(η,-1,1),(ξ,-1,1))

8⋅r₁⋅z₂ - 8⋅r₁⋅z₄ - 8⋅r₂⋅z₁ + 8⋅r₂⋅z₃ - 8⋅r₃⋅z₂ + 8⋅r₃⋅z₄ + 8⋅r₄⋅z₁ - 8⋅r₄⋅z₃

$$ \int_\Omega d\Omega = \frac{1}{2}\left((r_1-r_3)(z_2-z_4) +(r_2-r_4)(z_3-z_1)\right)$$

# 求解$ \int_\Omega u d\Omega$ 

In [51]:
@vars r₁ r₂ r₃ r₄ η ξ  z₁ z₂ z₃ z₄ r_ξ z_η r_η z_ξ u u₁ u₂ u₃ u₄

ξn = [-1,1,1,-1]
ηn = [-1,-1,1,1]
un = [u₁,u₂,u₃,u₄]
ϕ = [(1+ξn[i]*ξ)*(1+ηn[i]*η) for i in 1:4] 
r = [rn[i]*ϕ[i] for i in 1:4]
z =  [zn[i]*ϕ[i] for i in 1:4]

# r_ξ=0; z_η=0; r_η =0; z_ξ =0; u =0
# for i in 1:4
#     r_ξ += ξn[i]*r[i]*(1+ηn[i]*η)
#     z_η += ηn[i]*z[i]*(1+ξn[i]*ξ)
#     r_η += ηn[i]*r[i]*(1+ξn[i]*ξ)
#     z_ξ += ξn[i]*z[i]*(1+ηn[i]*η)
#     u   += un[i]*
# end
# y = r_ξ*z_η -r_η*z_ξ
# y = integrate(y,(η,-1,1),(ξ,-1,1))


4-element Array{Sym,1}:
 (1 - η)*(1 - ξ)
 (1 - η)*(ξ + 1)
 (η + 1)*(ξ + 1)
 (1 - ξ)*(η + 1)